# Compute the train set stats


In [1]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import json
import sys
import torch
import numpy as np
import random
import os

/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/limfuoen/anaconda3/envs/artificial-intelligence/bin/../lib/libjpeg.9.dylib' (no su

In [2]:
# Get the absolute path to the project root without relying on __file__
notebook_path = os.getcwd()  # Gets current working directory
project_root = os.path.abspath(
    os.path.join(notebook_path, "../cnn_scripts"))

sys.path.insert(0, project_root)

Set the seed for reproducability


In [3]:
random_seed = 42

torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

if (torch.cuda.is_available()):
    torch.cuda.manual_seed(random_seed)

Set some constant strings


In [4]:
STATS_FILE_NAME = 'train_set_stats.json'

train_csv = '../data/melspectrogram_train_dataset.csv'
root_dir = '../data/'

Custom dataset class for loading the mel spectrogram images


In [5]:
from datasets.MelspectrogramDataset import MelSpectrogramDataset

In [6]:
# Define a simple transform to apply ResizePad (if needed) and convert to tensor
# Replace ResizePad with your preferred transform if you use it
transform = transforms.Compose([
    # For example, if you use ResizePad:
    # ResizePad(size=(224, 224)),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = MelSpectrogramDataset(
    csv_file=train_csv, root_dir=root_dir, transform=transform)

In [7]:
def compute_mean_std(dataset, batch_size=16, num_workers=8, pin_memory=True):
    loader = DataLoader(dataset, batch_size=batch_size,
                        num_workers=num_workers, shuffle=False, pin_memory=pin_memory)
    mean = 0.0
    std = 0.0
    total_samples = 0

    for data, _ in loader:
        batch_samples = data.size(0)
        # Flatten the height and width dimensions into one
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        total_samples += batch_samples

    mean /= total_samples
    std /= total_samples
    return mean, std

In [8]:
# Compute the mean and standard deviation
mean, std = compute_mean_std(train_dataset, batch_size=16, num_workers=8)
print("Computed Mean:", mean)
print("Computed Std:", std)

# Save the computed stats to a JSON file
stats = {"mean": mean.tolist(), "std": std.tolist()}
with open(STATS_FILE_NAME, "w") as f:
    json.dump(stats, f)

print(f"Saved computed mean and std to {STATS_FILE_NAME}")

/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/limfuoen/anaconda3/envs/artificial-intelligence/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/limfuoen/anaconda3/envs/artificial-intelligence/bin/../lib/libjpeg.9.dylib' (no su

Computed Mean: tensor([0.2126, 0.0765, 0.1826])
Computed Std: tensor([0.2794, 0.1221, 0.1852])
Saved computed mean and std to train_set_stats.json
